In [1]:
#Importing the necessary libraries
#PolymerDomain -- determine the region of the systems where the polymer is located, and help determine the feed to the membrane
#os -- read and write files
#clear_output -- clear the print statements after each iteration

from obtain_polymer_domain import PolymerDomain
import os
from IPython.display import clear_output

In [2]:
#Creating an object of the class PolymerDomain
config=PolymerDomain()

In [ ]:
#File path to the simulation data
file_path="/scratch1/sbinkas/polyamide-sd/PADQI/Box-81/AfterCompression/nemd-equilibration"

#Initialize empty lists to store the values
box_x_configs=[] #x-dimension of the box for each config
box_y_configs=[] #y-dimension of the box for each config
min_poly_configs=[] #minimum z-dimension of the polymer domain for each config
max_poly_configs=[] #maximum z-dimension of the polymer domain for each config
feed_strip_configs=[] #feed water domain for each config
permeate_strip_configs=[] #permeate water domain for each config
polymer_dom_configs=[] #polymer domain for each config

#Loop through each configuration
# i -- looping thrugh the timestamps(in ns) from production run of dry polymer equilibration
# j -- looping through the timestamps(in ns) from NEMD run of polymer
# k -- looping through two random copies of each config

for i in range(100,301,40):
    for j in range(55,101,5):
        for k in range(1,2):
            print("Calculating for:",i,j,k)
            #Run PolymerDomain() to obtain the feed and permeate water molecules infomration
            geom_output=config.main(gro_file=os.path.join(file_path,"prod-"+str(i),"nemd-"+str(j)+"-ns-copy-"+str(k),"prod-padqi-npt-"+str(j)+"-ns-copy-"+str(k)+".gro"),
                                    o=os.path.join(file_path,file_path,"prod-"+str(i),"nemd-"+str(j)+"-ns-copy-"+str(k),"cut.gro"),
                                    dzp=0.6,
                                    dzn=0.6,
                                    n_atoms_water=4.0,
                                    bash=False,
                                    polymer_resnames="MPD TMC",
                                    solvent_resnames="SOL")
            #Clear the print statements after each iteration
            clear_output(wait=True)
            
            #Save the output values in the respective lists
            box_x_configs.append(geom_output['Box x(nm)'])
            box_y_configs.append(geom_output['Box y(nm)'])
            min_poly_configs.append(geom_output['Min polymer coordinate'])
            max_poly_configs.append(geom_output['Max polymer coordinate'])
            feed_strip_configs.append(geom_output['Feed strip water molecules'])
            permeate_strip_configs.append(geom_output['Permeate strip water molecules'])
            polymer_dom_configs.append(geom_output['Polymer water molecules'])

In [4]:
#Now save the lists in dataframe for easy visualization
import pandas as pd
configs_data=pd.DataFrame(list(zip(box_x_configs,
                                   box_y_configs,
                                   min_poly_configs,
                                   max_poly_configs,
                                   feed_strip_configs,
                                   permeate_strip_configs,
                                   polymer_dom_configs)),
                                   columns=["Box x(nm)","Box y(nm)","Polymer min z(nm)","Polymer max z(nm)","Feed strip","Permeate strip","Polymer domain"])
configs_data.describe()

,Box x(nm),Box y(nm),Polymer min z(nm),Polymer max z(nm),Feed strip,Permeate strip,Polymer domain
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000
mean,8.068705,7.919408,2.355600,8.942067,1283.416667,1282.316667,6593.833333
std,0.054529,0.053518,0.202655,0.173876,20.364157,21.046960,535.519167
min,7.982170,7.834480,1.731000,8.501000,1238.000000,1242.000000,5721.000000
25%,8.039127,7.890373,2.240250,8.825250,1271.250000,1265.000000,6113.750000
50%,8.061575,7.912415,2.348000,8.954500,1281.000000,1279.500000,6678.500000
75%,8.077015,7.927562,2.480750,9.037250,1291.750000,1295.250000,6988.750000
max,8.191360,8.039790,2.788000,9.398000,1329.000000,1332.000000,7680.000000


In [ ]:
polyamide_xy_area=configs_data["Box x(nm)"].mean()*configs_data["Box y(nm)"].mean()

hessam_xy_area=26.32903 #XY area of the system in nm^2 from Hessam and Haji-Akbari paper, maintained it here for consistency

conversion_factor=polyamide_xy_area/hessam_xy_area

print(conversion_factor)

In [ ]:
#Now obtain target feed and permeate water molecules such that we have same XY area as Hessam and Haji-Akbari paper

feed_water=3400*conversion_factor
filtrate_water=2320*conversion_factor

#Desired number of ions in the system is 1.8M
ions_each_type=feed_water*(95/3400)

print(feed_water,filtrate_water,ions_each_type)

In [ ]:
total_water_molecules=feed_water+filtrate_water+configs_data["Polymer domain"].mean()+2*ions_each_type
print(total_water_molecules)